# Pré-processamento de Dados
Este notebook realiza o pré-processamento dos dados, incluindo a leitura do arquivo Excel e a criação de um dicionário de classes.

In [5]:
import os
import pandas as pd

def class_grs(grs):
    if 8 <= grs <= 15:
        return 0  # novice
    elif 16 <= grs <= 23:
        return 1  # intermediate
    elif 24 <= grs <= 31:
        return 2  # proeficient
    elif 32 <= grs <= 40:
        return 3  # specialist
    else:
        return -1  # GRS not in range

def get_classes():
    with open("data/classes.txt") as fich_labels:
        labels = fich_labels.read().split()
        classes = dict(zip(labels, list(range(len(labels)))))
    return classes

def criar_excel_limpo(videos_path, excel_path, output_excel="OSATS_clean.xlsx"):
    """
    Adiciona novos dados ao arquivo Excel 'OSATS_clean.xlsx' sem sobrescrever o conteúdo existente.
    Cada vídeo aparece apenas uma vez, com a média das classificações calculada e a classe correspondente gerada.
    
    Args:
        videos_path (str): Caminho para a pasta contendo os vídeos.
        excel_path (str): Caminho para o arquivo OSATS.xlsx.
        output_excel (str): Nome do arquivo Excel de saída.
    """
    # Verificar se os caminhos existem
    if not os.path.exists(videos_path):
        raise FileNotFoundError(f"Pasta de vídeos não encontrada: {videos_path}")
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Arquivo Excel não encontrado: {excel_path}")
    
    # Carregar o arquivo Excel original
    df = pd.read_excel(excel_path)
    
    # Listar todos os arquivos de vídeo na pasta
    video_extensions = ('.mp4',)
    videos = [os.path.splitext(f)[0] for f in os.listdir(videos_path) if f.lower().endswith(video_extensions)]
    
    # Filtrar o DataFrame para conter apenas os vídeos presentes na pasta
    df_filtered = df[df['VIDEO'].isin(videos)]
    
    # Calcular a média do GLOBA_RATING_SCORE para cada vídeo
    df_mean = df_filtered.groupby('VIDEO')['GLOBA_RATING_SCORE'].mean().reset_index()
    
    # Adicionar a coluna 'GRS' com a classificação baseada na média
    df_mean['GRS'] = df_mean['GLOBA_RATING_SCORE'].apply(class_grs)

    df_mean = df_mean[df_mean['GRS'] != -1]
    
    # Selecionar apenas as colunas 'VIDEO' e 'GRS'
    df_clean = df_mean[['VIDEO', 'GRS']]
    
    # Verificar se o arquivo de saída já existe
    if os.path.exists(output_excel):
        # Carregar o conteúdo existente
        existing_df = pd.read_excel(output_excel)
        
        # Concatenar os novos dados com os existentes, evitando duplicatas
        combined_df = pd.concat([existing_df, df_clean]).drop_duplicates(subset=['VIDEO']).reset_index(drop=True)
    else:
        # Caso o arquivo não exista, usar apenas os novos dados
        combined_df = df_clean
    
    # Salvar o arquivo atualizado
    combined_df.to_excel(output_excel, index=False)
    print(f"Arquivo '{output_excel}' atualizado com sucesso!")

In [6]:
# Exemplo de uso
VIDEO_PATH = "data/cropped_videos"
EXCEL_PATH = "data/OSATS.xlsx"
criar_excel_limpo(VIDEO_PATH, EXCEL_PATH)
clean_excel_file = pd.read_excel("OSATS_clean.xlsx")
clean_excel_file

Arquivo 'OSATS_clean.xlsx' atualizado com sucesso!


,VIDEO,GRS
0,A31H,0
1,A41X,3
2,A59Z,0
3,A66S,2
4,A83X,2
...,...,...
184,Z49G,2
185,Z49X,1
186,Z62S,0
187,Z93E,2


In [10]:
import os
import pandas as pd

def class_grs(grs):
    if 8 <= grs <= 15:
        return 0  # novice
    elif 16 <= grs <= 23:
        return 1  # intermediate
    elif 24 <= grs <= 31:
        return 2  # proeficient
    elif 32 <= grs <= 40:
        return 3  # specialist
    else:
        return -1  # GRS not in range

def criar_excel_limpo_multicol(videos_path, excel_path, output_excel="OSATS_clean_multicol.xlsx"):
    """
    Cria um arquivo Excel limpo com a média das colunas OSATS para cada vídeo,
    arredondando as médias e aplicando class_grs apenas para GLOBA_RATING_SCORE.
    Só salva linhas onde todas as médias e a classe são válidas.
    """
    osats_cols = [
        'OSATS_RESPECT', 'OSATS_MOTION', 'OSATS_INSTRUMENT', 'OSATS_SUTURE',
        'OSATS_FLOW', 'OSATS_KNOWLEDGE', 'OSATS_PERFORMANCE', 'OSATS_FINAL_QUALITY', 'GLOBA_RATING_SCORE'
    ]
    
    # Verificar se os caminhos existem
    if not os.path.exists(videos_path):
        raise FileNotFoundError(f"Pasta de vídeos não encontrada: {videos_path}")
    if not os.path.exists(excel_path):
        raise FileNotFoundError(f"Arquivo Excel não encontrado: {excel_path}")
    
    # Carregar o arquivo Excel original
    df = pd.read_excel(excel_path)
    
    # Listar todos os arquivos de vídeo na pasta
    video_extensions = ('.mp4',)
    videos = [os.path.splitext(f)[0] for f in os.listdir(videos_path) if f.lower().endswith(video_extensions)]
    
    # Filtrar o DataFrame para conter apenas os vídeos presentes na pasta
    df_filtered = df[df['VIDEO'].isin(videos)]
    
    # Calcular a média das colunas OSATS para cada vídeo
    df_mean = df_filtered.groupby('VIDEO')[osats_cols].mean().reset_index()
    
    # Arredondar as médias para inteiro
    for col in osats_cols:
        df_mean[col] = df_mean[col].round().astype(int)
    
    # Aplicar class_grs apenas para GLOBA_RATING_SCORE
    df_mean['GRS'] = df_mean['GLOBA_RATING_SCORE'].apply(class_grs)
    
    # Remover linhas onde qualquer coluna ficou -1
    cols_to_check = osats_cols + ['GRS']
    df_clean = df_mean[(df_mean[cols_to_check] != -1).all(axis=1)]
    
    # Selecionar apenas VIDEO, médias e GRS
    df_final = df_clean[['VIDEO'] + osats_cols + ['GRS']]
    
    # Verificar se o arquivo de saída já existe
    if os.path.exists(output_excel):
        existing_df = pd.read_excel(output_excel)
        combined_df = pd.concat([existing_df, df_final]).drop_duplicates(subset=['VIDEO']).reset_index(drop=True)
    else:
        combined_df = df_final
    
    # Salvar o arquivo atualizado
    combined_df.to_excel(output_excel, index=False)
    print(f"Arquivo '{output_excel}' atualizado com sucesso!")

In [11]:
# Exemplo de uso
VIDEO_PATH = "data/cropped_videos"
EXCEL_PATH = "data/OSATS.xlsx"
criar_excel_limpo_multicol(VIDEO_PATH, EXCEL_PATH)
clean_excel_file_multi = pd.read_excel("OSATS_clean_multicol.xlsx")
clean_excel_file_multi

Arquivo 'OSATS_clean_multicol.xlsx' atualizado com sucesso!


,VIDEO,OSATS_RESPECT,OSATS_MOTION,OSATS_INSTRUMENT,OSATS_SUTURE,OSATS_FLOW,OSATS_KNOWLEDGE,OSATS_PERFORMANCE,OSATS_FINAL_QUALITY,GLOBA_RATING_SCORE,GRS
0,A31H,1,2,2,2,2,2,2,1,14,0
1,A41X,5,4,4,5,4,4,4,4,34,3
2,A59Z,2,1,2,1,1,1,1,1,11,0
3,A66S,3,3,3,3,3,4,3,3,26,2
4,A83X,4,2,4,3,3,4,4,3,27,2
...,...,...,...,...,...,...,...,...,...,...,...
201,Z49X,2,2,2,2,2,2,2,2,18,1
202,Z62S,2,1,1,1,1,1,1,1,10,0
203,Z93E,4,4,3,3,4,4,3,3,29,2
204,Z95A,4,2,3,3,2,3,3,3,23,1
